In [ ]:
import numpy as np
import pandas as pd
import math
from scipy.stats import loguniform
from statistics import mean
from scipy import stats
#import scikit_posthocs
import warnings 

from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, RandomizedSearchCV
from sklearn.metrics import classification_report,f1_score
from sklearn.impute import SimpleImputer 
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
#from scikit_posthocs import posthoc_nemenyi_friedman


warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('heart.csv')
df

In [ ]:
df.shape

In [ ]:
a=df.dtypes
a

In [ ]:
df.describe()
df

In [ ]:
#from the above there is no datatype object that condradicts with the above description

# we splitted the dataset based on feature type=categorical and numeric. 
allFeatures=df.columns[1:len(df.columns)-1]
catFeatures=[]
for i in range((len(df.columns)-1)):
    if df.dtypes[i]=='O':
        print(i)
        catFeatures.append(i)
catFeatures.append(5) #fasting bs is categorical 1: if FastingBS > 120 mg/dl, 0: otherwise

catFeatures=df.columns[catFeatures]
print(catFeatures)

numFeatures= [i for i in allFeatures if not(i in catFeatures)]
print(numFeatures)

In [ ]:
df[numFeatures].describe()
#get rid of min resting bp or data points that are insignificant.


In [ ]:
s=0
c= np.where(df['RestingBP']==0)
print(c[0])
df.iloc[449]

In [ ]:
df=df.drop(index=449,axis=0) #removed this index because it has resting BP. and Cholestrol zero


In [ ]:
df=df.reset_index()
df=df.drop(['index'],axis=1)
df

In [ ]:
plt.rcParams["figure.figsize"] = [8, 5]
plt.rcParams["figure.autolayout"] = True
df[numFeatures].hist()


In [ ]:

for cat in catFeatures:
    fig, ax = plt.subplots()
    df[cat].value_counts().plot(ax=ax, kind='bar', xlabel=cat, ylabel='frequency')
    plt.show()

In [ ]:
for i in catFeatures:
    a=df[i].describe()
    print('Describe for '+ str(i))
    print(a)

In [ ]:
df.isnull().sum()

In [ ]:
#Compute Pearson pairwise correlation of features
corr_matrix=df[numFeature].corr()
print(corr_matrix)

In [ ]:
#Visualize Pearson pairwise correlation of features
fig = plt.figure()
sns.set(rc = {'figure.figsize':(15,8)})
ax = sns.heatmap(corr_matrix,cmap='coolwarm', vmin=-1, vmax=1,annot = True)
plt.xticks(rotation=90)
ax.set_xticklabels(df.columns)
ax.set_yticklabels(df.columns)
plt.show()

In [ ]:
#Remove Cholesterol variable
df = df.drop(labels = "Cholesterol", axis = 1)
df.head()

In [ ]:
scalor=StandardScaler()

scalor.fit(df)
df[numFeatures]=scalor.transform(X_train[numFeatures])
df[numFeatures]=scalor.transform(X_test[numFeatures])